In [65]:
from mesa import Agent, Model

In [66]:
agent_expectations = {
    "Residents (Positive)": {"Transparency": 0.5, "Inclusivity": 0.5, "Accountability": 0.3, "Outcome Fairness": 0.5},
    "Residents (Neutral)": {"Transparency": 0.6, "Inclusivity": 0.6, "Accountability": 0.4, "Outcome Fairness": 0.4},
    "Residents (Negative)": {"Transparency": 0.8, "Inclusivity": 0.8, "Accountability": 0.7, "Outcome Fairness": 0.3},
    "NGOs": {"Transparency": 0.8, "Inclusivity": 0.8, "Accountability": 0.6, "Outcome Fairness": 0.3},
    "Marginalized Groups": {"Transparency": 0.7, "Inclusivity": 0.9, "Accountability": 0.4, "Outcome Fairness": 0.4}
}

# sequences
sequence_events = {
    "Maximal Effort Process": [
        "Early public consultation",
        "Excursion",
        "Consultation: technology",
        "Consultation: nuisance and health",
        "Consultation: ecological impacts",
        "Consultation: noise and safety",
        "Consultation: commercial developer",
        "Consultation: local ownership",
        "Consultation: financial participation",
        "Consultation: external safety",
        "Consultation: Board authority",
        "Policy proposal",
        "Policy revision",
        "Public consultation",
        "Final decision"
    ],
    "Moderate Effort Process": [
        "Early public consultation",
        "Excursion",
        "Consultation: nuisance and health",
        "Consultation: financial participation",
        "Consultation: Board authority",
        "Policy proposal",
        "Public consultation",
        "Policy revision",
        "Final decision"
    ],
    "Basic Effort Process": [
        "Excursion",
        "Consultation: technology",
        "Policy proposal",
        "Public consultation",
        "Final decision"
    ],
    "Minimal Effort Process": [
        "Policy proposal",
        "Final decision"
    ]
}

event_impact = {
    "Early public consultation": {"Transparency": 1, "Inclusivity": 1},
    "Excursion": {"Transparency": 0.8, "Inclusivity": 0.6},
    "Consultation: technology": {"Transparency": 0.9, "Inclusivity": 0.7},
    "Consultation: nuisance and health": {"Transparency": 0.8, "Inclusivity": 0.9},
    "Consultation: ecological impacts": {"Transparency": 0.9, "Inclusivity": 0.8},
    "Consultation: noise and safety": {"Transparency": 0.9, "Inclusivity": 0.8},
    "Consultation: commercial developer": {"Transparency": 0.7, "Inclusivity": 0.6},
    "Consultation: local ownership": {"Inclusivity": 1, "Outcome Fairness": 0.8},
    "Consultation: financial participation": {"Outcome Fairness": 1, "Inclusivity": 0.8},
    "Consultation: external safety": {"Transparency": 0.9, "Accountability": 0.6},
    "Consultation: Board authority": {"Accountability": 0.8, "Transparency": 0.7},
    "Policy proposal": {"Accountability": 0.6},
    "Policy revision": {"Accountability": 1, "Outcome Fairness": 1},
    "Public consultation": {"Transparency": 1, "Inclusivity": 1},
    "Final decision": {"Outcome Fairness": 0.7},
}


In [67]:
# stakholder agent class

class StakeholderAgent(Agent):
    def __init__(self, unique_id, model, stakeholder_type):
        super().__init__(model)
        self.unique_id = unique_id
        self.stakeholder_type = stakeholder_type
        self.perceptions = {}
        self.reactions = {}

    def calculate_perceptions(self):
        expectations = agent_expectations[self.stakeholder_type]
        max_events = max(len(events) for events in self.model.sequence_events.values())

        for sequence, events in self.model.sequence_events.items():
            actual_scores = {"Transparency": 0, "Inclusivity": 0, "Accountability": 0, "Outcome Fairness": 0}
            dimension_counts = {"Transparency": 0, "Inclusivity": 0, "Accountability": 0, "Outcome Fairness": 0}

            # sum impacts per dimension
            for event in events:
                impacts = self.model.event_impact.get(event, {})
                for dim in impacts:
                    actual_scores[dim] += impacts[dim]
                    dimension_counts[dim] += 1

            # penalties for missing policy proposal and/or revision
            if "Policy proposal" in events:
                proposal_index = events.index("Policy proposal")
                found_consultation = False
                for e in events[:proposal_index]:
                    if e == "Public consultation" or e == "Early public consultation":
                        found_consultation = True
                if not found_consultation:
                    actual_scores["Transparency"] -= 0.5
                    actual_scores["Inclusivity"] -= 0.5

            if "Policy revision" not in events:
                actual_scores["Accountability"] -= 0.5
                actual_scores["Outcome Fairness"] -= 0.5

            # average dimension scores, penalize missing strongly
            addressed_dims = 0
            for dim in actual_scores:
                if dimension_counts[dim] > 0:
                    actual_scores[dim] /= dimension_counts[dim]
                    addressed_dims += 1
                else:
                    actual_scores[dim] = 0  # penalty for unaddressed dimensions

                # ensure range from 0 to 1
                actual_scores[dim] = max(0, min(actual_scores[dim], 1))

            # base justice calculation, difference between actual and expected for each dimension
            squared_diff = sum((actual_scores[dim] - expectations[dim]) ** 2 for dim in expectations)
            max_diff = len(expectations)
            justice_score = 1 - (squared_diff / max_diff)

            # coverage factor (number of addressed dimensions)
            coverage_factor = addressed_dims / len(expectations)

            # sequence length factor (penalize shorter sequences)
            sequence_length_factor = len(events) / max_events

            # score is a product of the base score and the two factors 
            justice_score *= coverage_factor * sequence_length_factor
            justice_score = round(max(0, min(justice_score, 1)), 3)

            self.perceptions[sequence] = justice_score

            if justice_score >= 0.7:
                self.reactions[sequence] = "Support"
            elif justice_score >= 0.4:
                self.reactions[sequence] = "Neutral"
            else:
                self.reactions[sequence] = "Oppose"

    # model class
    
    class JusticeModel(Model):
        def __init__(self):
            super().__init__()
            self.stakeholders = []
            self.sequence_events = sequence_events
            self.event_impact = event_impact

            agent_types = list(agent_expectations.keys())
            for i, name in enumerate(agent_types):
                agent = StakeholderAgent(i, self, name)
                self.stakeholders.append(agent)

        def step(self):
            for agent in self.stakeholders:
                agent.calculate_perceptions()


In [68]:
# run model
model = JusticeModel()
model.step()

# output
for agent in model.stakeholders:
    print(f"\n{agent.stakeholder_type}")
    for sequence in sequence_events:
        score = agent.perceptions[sequence]
        reaction = agent.reactions[sequence]
        print(f"  {sequence}: score = {score}, reaction = ({reaction})")


Residents (Positive)
  Maximal Effort Process: score = 0.856, reaction = (Support)
  Moderate Effort Process: score = 0.5, reaction = (Neutral)
  Basic Effort Process: score = 0.317, reaction = (Oppose)
  Minimal Effort Process: score = 0.056, reaction = (Oppose)

Residents (Neutral)
  Maximal Effort Process: score = 0.884, reaction = (Support)
  Moderate Effort Process: score = 0.518, reaction = (Neutral)
  Basic Effort Process: score = 0.321, reaction = (Oppose)
  Minimal Effort Process: score = 0.052, reaction = (Oppose)

Residents (Negative)
  Maximal Effort Process: score = 0.916, reaction = (Support)
  Moderate Effort Process: score = 0.543, reaction = (Neutral)
  Basic Effort Process: score = 0.299, reaction = (Oppose)
  Minimal Effort Process: score = 0.039, reaction = (Oppose)

NGOs
  Maximal Effort Process: score = 0.911, reaction = (Support)
  Moderate Effort Process: score = 0.539, reaction = (Neutral)
  Basic Effort Process: score = 0.308, reaction = (Oppose)
  Minimal Ef